In [2]:
import sqlalchemy
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [84]:
engine = sqlalchemy.create_engine('sqlite:///{}'.format(os.path.join(os.getcwd(), 'imdb_test.db')))
movies = pd.read_sql("""select * from movies""", con=engine)

In [4]:
movies.head()

,id,Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,...,Response,Runtime,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes
0,None,"Peter Weller, Nancy Allen, Dan O'Herlihy, Ronn...",Nominated for 2 Oscars. Another 11 wins & 10 n...,N/A,USA,02 Oct 2001,Paul Verhoeven,"Action, Crime, Sci-Fi",English,67,...,True,102 min,RoboCop,movie,https://www.facebook.com/RobocopMovie,"Edward Neumeier, Michael Miner",1987,tt0093870,7.5,"204,183"
1,None,"Rachel Weisz, Vanessa Redgrave, Monica Bellucc...",8 wins & 11 nominations.,"$900,000","Canada, Germany",24 Jan 2012,Larysa Kondracki,"Action, Biography, Crime","English, Romanian, Russian, Serbian",59,...,True,112 min,The Whistleblower,movie,http://www.thewhistleblower-movie.com/,"Larysa Kondracki, Eilis Kirwan",2010,tt0896872,7.2,"28,522"
2,None,"Yoon-Seok Kim, Jung-woo Ha, Yeong-hie Seo, Yoo...",18 wins & 24 nominations.,N/A,South Korea,04 Aug 2009,Hong-jin Na,"Action, Crime, Thriller",Korean,64,...,True,125 min,The Chaser,movie,http://www.thechaser.co.kr/,"Won-Chan Hong, Shinho Lee, Hong-jin Na",2008,tt1190539,7.9,"46,084"
3,None,"Kevin Conroy, Dana Delany, Hart Bochner, Stacy...",1 nomination.,N/A,USA,21 Dec 1999,"Eric Radomski, Bruce Timm, Kevin Altieri, Boyd...","Animation, Action, Adventure",English,N/A,...,True,76 min,Batman: Mask of the Phantasm,movie,N/A,"Bob Kane (character created by: Batman), Alan ...",1993,tt0106364,7.9,"35,653"
4,None,"Maggie Cheung, Tony Chiu-Wai Leung, Ping Lam S...",Nominated for 1 BAFTA Film Award. Another 44 w...,N/A,"Hong Kong, China",05 Mar 2002,Kar-Wai Wong,"Drama, Romance","Cantonese, Shanghainese, French, Spanish",85,...,True,98 min,In the Mood for Love,movie,http://www.wkw-inthemoodforlove.com,Kar-Wai Wong,2000,tt0118694,8.1,"98,079"


In [8]:
movies.columns

Index(['id', 'Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director',
       'Genre', 'Language', 'Metascore', 'Plot', 'Poster', 'Production',
       'Rated', 'Released', 'Response', 'Runtime', 'Title', 'Type', 'Website',
       'Writer', 'Year', 'imdbID', 'imdbRating', 'imdbVotes'],
      dtype='object')

# extract year from 'Released'

In [46]:
movies['Released_year'] = movies['Released'].str.extract(r'\w (\d+)').astype(float)
movies.query('Released_year != Year')[['Released', 'Released_year', 'Year']].head()

/home/ariel/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,Released,Released_year,Year
1,27 Oct 2011,2011.0,2010
4,09 Mar 2001,2001.0,2000
10,28 Jan 2005,2005.0,2004
15,10 May 1967,1967.0,1965
16,27 Aug 2004,2004.0,2002


In [38]:
np.abs(movies['Released_year'] - movies['Year']).value_counts()

0.0     558
1.0      83
2.0      14
6.0       1
4.0       1
19.0      1
dtype: int64

The difference is not so big so I'll stick to the 'Year' column

In [47]:
movies.drop('Released_year', axis=1, inplace=True)

# clean

In [95]:
def clean_movies(movies_df):
    movies_df = movies_df.copy()
    # drop unrelevant columns
    movies_df.drop(['id', 'DVD', 'Website', 'Response', 'Poster', 'Released'], axis=1, inplace=True)
    # conver to nan
    movies_df.replace('N/A', np.nan, inplace=True)
    
    movies_df['imdbVotes'].str.replace(',', '').astype(float)
    
    BoxOffice_pound_index = movies_df.dropna()[~movies_df['BoxOffice'].dropna().str.contains('\$')].index
    movies_df['BoxOffice'] = movies_df.BoxOffice.str.replace('\D', '').astype(float)
    movies_df.loc[BoxOffice_pound_index, 'BoxOffice'] *= 1.3
    return movies_df

In [92]:
movies[~movies['BoxOffice'].str.contains('\$')].index

Int64Index([  0,   2,   3,   4,   9,  11,  15,  16,  17,  18,
            ...
            640, 641, 642, 646, 650, 655, 658, 660, 661, 662],
           dtype='int64', length=395)

In [96]:
clean = clean_movies(movies)

/home/ariel/venv/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [97]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663 entries, 0 to 662
Data columns (total 19 columns):
Actors        663 non-null object
Awards        579 non-null object
BoxOffice     283 non-null float64
Country       663 non-null object
Director      663 non-null object
Genre         663 non-null object
Language      662 non-null object
Metascore     532 non-null float64
Plot          663 non-null object
Production    640 non-null object
Rated         634 non-null object
Runtime       641 non-null object
Title         663 non-null object
Type          663 non-null object
Writer        663 non-null object
Year          663 non-null int64
imdbID        663 non-null object
imdbRating    636 non-null float64
imdbVotes     636 non-null object
dtypes: float64(3), int64(1), object(15)
memory usage: 98.5+ KB


In [98]:
clean.head()

,Actors,Awards,BoxOffice,Country,Director,Genre,Language,Metascore,Plot,Production,Rated,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Peter Weller, Nancy Allen, Dan O'Herlihy, Ronn...",Nominated for 2 Oscars. Another 11 wins & 10 n...,NaN,USA,Paul Verhoeven,"Action, Crime, Sci-Fi",English,67.0,"In a dystopic and crime-ridden Detroit, a term...",Orion Pictures Corporation,R,102 min,RoboCop,movie,"Edward Neumeier, Michael Miner",1987,tt0093870,7.5,"204,183"
1,"Rachel Weisz, Vanessa Redgrave, Monica Bellucc...",8 wins & 11 nominations.,900000.0,"Canada, Germany",Larysa Kondracki,"Action, Biography, Crime","English, Romanian, Russian, Serbian",59.0,A drama based on the experiences of Kathryn Bo...,IDP/Samuel goldwyn Films,R,112 min,The Whistleblower,movie,"Larysa Kondracki, Eilis Kirwan",2010,tt0896872,7.2,"28,522"
2,"Yoon-Seok Kim, Jung-woo Ha, Yeong-hie Seo, Yoo...",18 wins & 24 nominations.,NaN,South Korea,Hong-jin Na,"Action, Crime, Thriller",Korean,64.0,A disgraced ex-policeman who runs a small ring...,IFC Films,NOT RATED,125 min,The Chaser,movie,"Won-Chan Hong, Shinho Lee, Hong-jin Na",2008,tt1190539,7.9,"46,084"
3,"Kevin Conroy, Dana Delany, Hart Bochner, Stacy...",1 nomination.,NaN,USA,"Eric Radomski, Bruce Timm, Kevin Altieri, Boyd...","Animation, Action, Adventure",English,NaN,Batman is wrongly implicated in a series of mu...,WARNER BROTHERS PICTURES,PG,76 min,Batman: Mask of the Phantasm,movie,"Bob Kane (character created by: Batman), Alan ...",1993,tt0106364,7.9,"35,653"
4,"Maggie Cheung, Tony Chiu-Wai Leung, Ping Lam S...",Nominated for 1 BAFTA Film Award. Another 44 w...,NaN,"Hong Kong, China",Kar-Wai Wong,"Drama, Romance","Cantonese, Shanghainese, French, Spanish",85.0,"Two neighbors, a woman and a man, form a stron...",USA Films,PG,98 min,In the Mood for Love,movie,Kar-Wai Wong,2000,tt0118694,8.1,"98,079"


In [99]:
clean.loc[72, 'BoxOffice']

22133252.7

In [81]:
clean['BoxOffice'].dropna()[~clean['BoxOffice'].dropna().str.contains('\$')]

72      &pound;17,025,579
95       &pound;4,676,810
110    &pound;105,996,316
117     &pound;17,000,000
190        &pound;584,553
198      &pound;1,612,881
233        &pound;100,000
236        &pound;114,501
249      &pound;2,317,507
329      &pound;2,425,377
447         &pound;39,567
467        &pound;296,294
470    &pound;393,201,353
508    &pound;664,987,816
517     &pound;89,021,735
Name: BoxOffice, dtype: object

In [63]:
v =[]
for i in movies['BoxOffice'].str.findall(r'\D'):
    v += i
set(v)

{'$', '&', ',', '/', ';', 'A', 'N', 'd', 'n', 'o', 'p', 'u'}

In [1]:
# movies.to_sql(name='movies_clean', con=engine)

NameError: name 'movies' is not defined